In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Core column and expression functions
from pyspark.sql.functions import col, lit, when, lag, datediff

# Date-related functions
from pyspark.sql.functions import year, month

# Aggregation functions
from pyspark.sql.functions import sum as _sum, avg, count, rank

In [0]:
pde_silver = spark.read.parquet("/mnt/silver/pde/pde_silver")

Top drugs by total spend

In [0]:
from pyspark.sql.functions import sum as _sum, count

top_drugs = pde_silver.groupBy("prod_srvc_id").agg(
    count("pde_id").alias("prescription_count"),
    _sum("tot_rx_cst_amt").alias("total_cost")
).orderBy("total_cost", ascending=False)

top_drugs.write.mode("overwrite").parquet("/mnt/gold/pde/top_drugs_by_cost")


In [0]:
top_drugs.show()

Patient spend summary

In [0]:
patient_spend = pde_silver.groupBy("desynpuf_id").agg(
    _sum("ptnt_pay_amt").alias("total_patient_payment"),
    _sum("tot_rx_cst_amt").alias("total_drug_cost")
)

patient_spend.write.mode("overwrite").parquet("/mnt/gold/pde/patient_spend_summary")


In [0]:
patient_spend.show()

Prescription activity over time

In [0]:
daily_rx_trend = pde_silver.groupBy("srvc_dt").agg(
    count("pde_id").alias("prescription_count"),
    _sum("tot_rx_cst_amt").alias("total_cost")
).orderBy("srvc_dt")

daily_rx_trend.write.mode("overwrite").parquet("/mnt/gold/pde/prescription_trends")


In [0]:
daily_rx_trend.show()